In [1]:
import sys
import importlib
import logging
import numpy as np

sys.path.append("./src/")
import pdb_numpy
import pdb_numpy.alignement as alignement
import pdb_numpy.DSSP as DSSP


In [2]:
from pdb_numpy.data.blosum import BLOSUM62
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

def print_align_seq(seq_1, seq_2, line_len=80):
    """Print the aligned sequences with a line length of 80 characters.

    Parameters
    ----------
    seq_1 : str
        First sequence
    seq_2 : str
        Second sequence
    line_len : int, optional
        Length of the line, by default 80

    Returns
    -------
    None

    """

    sim_seq = ""
    for i in range(len(seq_1)):

        if seq_1[i] == seq_2[i]:
            sim_seq += "*"
            continue
        elif seq_1[i] != "-" and seq_2[i] != "-":
            if (seq_1[i], seq_2[i]) in BLOSUM62:
                mut_score = BLOSUM62[seq_1[i], seq_2[i]]
            else:
                mut_score = BLOSUM62[seq_2[i], seq_1[i]]
            if mut_score >= 0:
                sim_seq += "|"
                continue
        sim_seq += " "

    for i in range(1 + len(seq_1) // line_len):
        print(seq_1[i * line_len : (i + 1) * line_len])
        print(sim_seq[i * line_len : (i + 1) * line_len])
        print(seq_2[i * line_len : (i + 1) * line_len])
        print("\n")

    identity = 0
    similarity = 0
    for char in sim_seq:
        if char == "*":
            identity += 1
        if char in ["|", "*"]:
            similarity += 1

    len_1 = len(seq_1.replace("-", ""))
    len_2 = len(seq_2.replace("-", ""))

    print(f"Identity seq1: {identity / len_1 * 100:.2f}%")
    print(f"Identity seq2: {identity / len_2 * 100:.2f}%")

    print(f"Similarity seq1: {similarity / len_1 * 100:.2f}%")
    print(f"Similarity seq2: {similarity / len_2 * 100:.2f}%")

    return



In [3]:

def align_seq(seq_1, seq_2, gap_cost=-11, gap_extension=-1):
    """Align two amino acid sequences using the Waterman - Smith Algorithm.

    Parameters
    ----------
    seq_1 : str
        First sequence to align
    seq_2 : str
        Second sequence to align
    gap_cost : int, optional
        Cost of gap, by default -8
    gap_extension : int, optional
        Cost of gap extension, by default -2

    Returns
    -------
    str
        Aligned sequence 1
    str
        Aligned sequence 2
    """

    seq_1 = seq_1.replace("-", "")
    seq_2 = seq_2.replace("-", "")

    len_1 = len(seq_1)
    len_2 = len(seq_2)

    print(len_1, len_2)

    # Initialize the matrix
    matrix = np.zeros((len_1 + 1, len_2 + 1), dtype=int)

    prev_line = np.zeros((len_2 + 1), dtype=bool)
    choices = np.zeros((3), dtype=int)

    # Fill the matrix
    for i in range(1, len_1 + 1):
        # print(i)
        prev = False  # insertion matrix[i, j - 1]
        for j in range(1, len_2 + 1):
            # Identify the BLOSUM62 score
            # Match
            choices[0] = matrix[i - 1, j - 1] + BLOSUM62[(seq_2[j - 1], seq_1[i - 1])]
            # Delete
            choices[1] = matrix[i - 1, j] + (gap_extension if prev else gap_cost)
            # Insert
            choices[2] = matrix[i, j - 1] + (gap_extension if prev_line[j] else gap_cost)

            max_index = np.argmax(choices)
            matrix[i, j] = choices[max_index]
            prev_line[j] = False
            prev = False

            if max_index == 1:
                prev = True
            elif max_index == 2:
                prev_line[j] = True

    # Identify the maximum score
    min_seq = min(len_1, len_2)
    max_score = np.max(matrix[min_seq:, min_seq:])
    max_index = np.where(matrix == max_score)

    show_num = 15
    print(matrix[:show_num, :show_num])
    print(matrix[-show_num:, -show_num:])
    print(matrix[min_seq:, min_seq:])

    print("Max score:", max_score, max_index)


    index_list = []
    for i in range(len(max_index[0])):
        if max_index[0][i] >= min_seq and max_index[1][i] >= min_seq:
            index_list.append([max_index[0][i], max_index[1][i]])

    # if len(index_list) > 1:
    #    logger.warning(f"Ambigous alignement, {len(index_list)} solutions exists")

    i = index_list[0][0]
    j = index_list[0][1]
    print('Starting point:', i, j)
    # Traceback and compute the alignment
    align_1 = ""
    align_2 = ""

    if i != len_1:
        align_2 = (len_1 - i) * "-"

    if j != len_2:
        align_1 = (len_2 - j) * "-"

    align_1 += seq_1[i:]
    align_2 += seq_2[j:]

    i -= 1
    j -= 1

    while i != 0 and j != 0:
        if (
            matrix[i+1, j+1]
            == matrix[i, j] + BLOSUM62[(seq_1[i], seq_2[j])]
        ):
            align_1 = seq_1[i - 1] + align_1
            align_2 = seq_2[j - 1] + align_2
            i -= 1
            j -= 1
        elif (
            matrix[i+1, j+1] == matrix[i, j+1] + gap_cost
            or matrix[i+1, j+1] == matrix[i, j+1] + gap_extension
        ):
            align_1 = seq_1[i - 1] + align_1
            align_2 = "-" + align_2
            i -= 1
        else:
            # matrix[i+1, j+1] == matrix[i+1, j] + gap_cost
            #or matrix[i+1, j+1] == matrix[i+1, j] + gap_extension
            # ):
            align_1 = "-" + align_1
            align_2 = seq_2[j - 1] + align_2
            j -= 1

    align_1 = seq_1[:i] + align_1
    align_2 = seq_2[:j] + align_2

    if i != 0:
        align_2 = i * "-" + align_2
    elif j != 0:
        align_1 = j * "-" + align_1

    assert len(align_1) == len(align_2)

    return align_1, align_2


In [4]:
seq_1 = (
        "AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV"
        "RSGVRVKTYEPEAIWIPEIRFVNVENARDADVVDISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSV"
        "DTRNIVLAVDLEKVGKNDDVFLTGWDIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISW"
        "TAFWSTSYEANVTLVVSTLIAHIAFNILVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYLKVESQPARAASI"
        "TRASRIAFPVVFLLANIILAFLFFGF"
    )
seq_2 = (
        "APSEFLDKLMGKVSGYDARIRPNFKGPPVNVTCNIFINSFGSIAETTMDYRVNIFLR"
        "QQWNDPRLAYSEYPDDSLDLDPSMLDSIWKPDLFFANEKGANFHEVTTDNKLLRISKNGNVLYSIRITLVLACPMDLK"
        "NFPMDVQTCIMQLESFGYTMNDLIFEWDEKGAVQVADGLTLPQFILKEEKDLRYCTKHYNTGKFTCIEARFHLERQMG"
        "YYLIQMYIPSLLIVILSWVSFWINMDAAPARVGLGITTVLTMTTQSSGSRASLPKVSYVKAIDIWMAVCLLFVFSALL"
        "EYAAVNFIARAGTKLFISRAKRIDTVSRVAFPLVFLIFNIFYWITYKLVPR"
    )
%timeit align_seq_1, align_seq_2 = align_seq(seq_1, seq_2)


317 342
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   4  -1   1  -1  -2  -1  -2  -1  -1  -1   0  -1   0   1]
 [  0  -1   3  -1   3  -4  -4  -1  -1  -3  -1  -3   1  -3   0]
 [  0  -2  -2   3   1   0  -8   2  -2  -5  -6  -2  -4  -2  -3]
 [  0  -1  -4  -3   1   1   2  -9   1   0   0  -9  -3  -3  -3]
 [  0   0  -3  -6  -5   0   2  -1 -10   2   1  -3 -11   1  -5]
 [  0   1  -1   1  -6  -7  -2   2  -1  -9   1   1  -3 -10   5]
 [  0  -1   8  -2   0 -10 -10  -3   1  -4 -10   0   0  -5  -6]
 [  0  -1   6   7  -3  -4 -13 -11  -4  -2  -6 -11  -1  -2  -6]
 [  0  -1   6   5   6  -5  -7 -14 -12  -7  -4  -8 -12  -3  -3]
 [  0  -1   6   5   4   3  -8  -8 -15 -15  -9  -6  -9 -14  -4]
 [  0  -1  -4   4   2   4   5  -6 -11 -13 -14 -13  -9  -6 -15]
 [  0   4  -2  -3   3   0   3   3  -7 -12 -14 -14 -14  -9  -5]
 [  0  -2   3  -2  -1   0  -4   9   2  -9 -15 -15 -15 -17  -9]
 [  0  -1  -3   3   3  -4  -3  -2  10   9  -2 -13 -14 -17 -17]]
[[312 319 318 311 316 315 314 313 312 311 310 

In [5]:
%timeit align_seq_1, align_seq_2 = alignement.align_seq_C(seq_1, seq_2)


1.85 ms ± 370 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
print_align_seq(align_seq_1, align_seq_2, line_len=80)

--------------------AQDMVSPPPPI-ADEPLTVN-TGIYLIECYSLDDKAETFKVNAF-LSLSWKDRRLAFDP-
                      || |**  | |   | |*             | |  ||** *    |*|* ***||  
APSEFLDKLMGKVSGYDARIRPNFKGPPVNVTC--NIFINSF--GS-I-A---ETTMDYRVNIFLR-QQWNDPRLAYSEY


VRSGVRVKTYEPEAIWIPE-IRFVNVENA-RD---ADVVDISVSPDGTVQY-LERFSARVLSPLDFRRYPF--D--SQTL
  ||| |     ||** *|   *|* ||*      |*   | |* |*|* *   *||  |  *|*||||*|  |   | *
PDDSLDLDPSMLDSIWKPDL-FFANEKGANFHEVTTDNKLLRISKNGNVLYS-IRITLVLACPMDLKNFPMDVQTCIMQL


-H-I-Y--LI-V----R-SV-DTRNIVLAVDLEKVGKNDDVFLT-GWDIESFTAVVKPANFALEDRLESKLDYQLRISRQ
         |      | |*  | |||*   | *   |*  | *  ||  |**  |       | |** || *     ||
ESFGYTMNDLIFEWDEKGAVQVADGLTLPQFILKEE-KDLRYCTKHYNTGKFT-CI----E-ARFHLERQMGYY-L-IQM


YFSYIPNIILPMLFILFISWTAF-W-STSYEANVTLVVSTLIAHIA-FNILVETNLPKTPYMTYTGAI-I-F-MIYLF-Y
   ***     | |*|||**||*     |  *|* * ||*||  ||  |   |||***   ||*|      |  | *|  
---YIP--S--L-LIVILSWVSFWINMDAAPARVGLGITTVL-TMTTQSSGSRASLPK---VSYVK-AIDIWMAVCLLFV


FVAVI-EVTVQHY-LKVES-

In [29]:
from pdb_manip_py import pdb_manip

%timeit align_seq_1, align_seq_2 = pdb_manip.Coor.align_seq(seq_1, seq_2)
print_align_seq(align_seq_1, align_seq_2, line_len=80)

172 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPVRSGVRVKTYEPEAIWIPEIRFVN
                                                                                
--------------------------------------------------------------------------------


VENARDADVVDISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSVDTRNIVLAVDLEKVGKNDDVFLTGW
                                                                                
--------------------------------------------------------------------------------


DIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISWTAFWSTSYEANVTLVVSTLIAHIAFNI
                                                                                
--------------------------------------------------------------------------------


LVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYLKVESQPARAASITRASRIAFPVVFLLANIILAFLFFG
                       *****************************************************
-----------------------LFYFVAVIEVTVQHYLKV

In [27]:
seq_1 = 'AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV\
RSGVRVKTYEPEAIWIPEIRFVNVENARDADVVDISVSPDGTVQYLERFSARVLSPLDFRRYPFDSQTLHIYLIVRSV\
DTRNIVLAVDLEKVGKNDDVFLTGWDIESFTAVVKPANFALEDRLESKLDYQLRISRQYFSYIPNIILPMLFILFISW\
TAFWSTSYEANVTLVVSTLIAHIAFNILVETNLPKTPYMTYTGAIIFMIYLFYFVAVIEVTVQHYLKVESQPARAASI\
TRASRIAFPVVFLLANIILAFLFFGF'
seq_2 = 'LFYFVAVIEVTVQHYLKVESQPARAASITRASRIAFPVVFLLANIILAFLFFGF'
%timeit align_seq_1, align_seq_2 = alignement.align_seq(seq_1, seq_2)
alignement.print_align_seq(align_seq_1, align_seq_2, line_len=80)

[[  0   0   0   0   0   0   0   0   0   0]
 [  0  -1  -2  -2  -2   0   4   0  -1  -1]
 [  0  -2  -4  -3  -5  -4  -1   2  -3   1]
 [  0  -4  -5  -7  -6  -8  -6  -4  -1  -1]
 [  0   2  -4  -6  -7  -5  -9  -5  -3  -3]
 [  0   1   1  -5  -7  -3  -5  -5  -2  -5]
 [  0  -2  -1  -1  -7  -9  -2  -7  -7  -2]
 [  0  -3  -6  -4  -5  -9 -10  -4 -10  -8]
 [  0  -3  -7  -9  -8  -7 -10 -12  -7 -11]
 [  0  -3  -7 -10 -13 -10  -8 -12 -15  -8]]
Max score: 262 (array([317]), array([54]))
[[  0   0   0   0   0   0   0   0   0   0]
 [  0  -1  -2  -2  -2   0   4   0  -1  -1]
 [  0  -2  -4  -3  -5  -4  -1   2  -3   1]
 [  0  -4  -5  -7  -6  -8  -6  -4  -1  -1]
 [  0   2  -4  -6  -7  -5  -9  -5  -3  -3]
 [  0   1   1  -5  -7  -3  -5  -5  -2  -5]
 [  0  -2  -1  -1  -7  -9  -2  -7  -7  -2]
 [  0  -3  -6  -4  -5  -9 -10  -4 -10  -8]
 [  0  -3  -7  -9  -8  -7 -10 -12  -7 -11]
 [  0  -3  -7 -10 -13 -10  -8 -12 -15  -8]]
Max score: 262 (array([317]), array([54]))
[[  0   0   0   0   0   0   0   0   0   0]
 [  0  -1

In [103]:
seq_1 = 'AQDMVSP'
seq_2 = 'AQDMVSPPPPIA'

from pdb_numpy.data.blosum import BLOSUM62

def align_seq_DTW(seq_1, seq_2):
    """ Align two amino acid sequences using the Dynamic Time Warping Algorithm.
    """

    seq_1 = seq_1.replace("-", "")
    seq_2 = seq_2.replace("-", "")

    len_1 = len(seq_1)
    len_2 = len(seq_2)

    # Initialize the matrix
    matrix = np.zeros((len_1 + 1, len_2 + 1))
    matrix[1:, 0] = -np.inf
    matrix[0, 1:] = -np.inf

    direction = np.zeros((len_1 + 1, len_2 + 1))

    choices = np.zeros(3)

    # Fill the matrix
    for i in range(1, len_1 + 1):
        # print(i)
        prev = False  # insertion matrix[i, j - 1]
        for j in range(1, len_2 + 1):
            # Identify the BLOSUM62 score
            # Match
            choices = np.array([matrix[i - 1, j - 1], matrix[i - 1, j],  matrix[i, j - 1]])

            max_index = np.argmax(choices)
            #print(max_index)
            matrix[i, j] = choices[max_index] + BLOSUM62[(seq_2[j - 1], seq_1[i - 1])]
            direction[i, j] = max_index
    
    print(matrix)
    print(direction)

    i = len_1
    j = len_2

    align_1 = ""
    align_2 = ""

    while i != 0 and j != 0:
        if (
            direction[i, j] == 0
        ):
            align_1 = seq_1[i - 1] + align_1
            align_2 = seq_2[j - 1] + align_2
            i -= 1
            j -= 1
        elif (
            direction[i, j] == 1
        ):
            align_1 = seq_1[i - 1] + align_1
            align_2 = "-" + align_2
            i -= 1
        elif (
            direction[i, j] == 2
        ):
            align_1 = "-" + align_1
            align_2 = seq_2[j - 1] + align_2
            j -= 1

    i = len(align_1) - 1
    while i > 0:
        if (align_1[i] == "-" and align_2[i -1] == "-"):
            align_1 = align_1[:i] + align_1[i + 1:]
            align_2 = align_2[:i-1] + align_2[i:]
        elif (align_2[i] == "-" and align_1[i -1] == "-"):
            align_2 = align_2[:i] + align_2[i + 1:]
            align_1 = align_1[:i-1] + align_1[i:]
        i -= 1

    assert len(align_1) == len(align_2)

    print(align_1)
    print(align_2)

    return align_1, align_2

    

In [126]:
seq_1 = "AQDMVSPPPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV"
seq_2 = "AQDMVSPPPPIADEPLTVN"
align_seq_DTW(seq_1, seq_2)

[[  0. -inf -inf ... -inf -inf -inf]
 [-inf   4.   3. ...  -5.  -5.  -7.]
 [-inf   3.   9. ...  -2.  -4.  -4.]
 ...
 [-inf -29. -25. ...  95.  96. 100.]
 [-inf -30. -26. ...  94.  94.  98.]
 [-inf -30. -28. ...  94.  98.  95.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 2. ... 2. 2. 2.]
 [0. 1. 0. ... 2. 2. 2.]
 ...
 [0. 1. 1. ... 1. 1. 0.]
 [0. 1. 1. ... 1. 1. 1.]
 [0. 1. 1. ... 1. 0. 1.]]
AQDMVSP--PPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV
AQDMVSPPPP--IADEPLT----------------------------------V--N--


('AQDMVSP--PPPIADEPLTVNTGIYLIECYSLDDKAETFKVNAFLSLSWKDRRLAFDPV',
 'AQDMVSPPPP--IADEPLT----------------------------------V--N--')

In [127]:
import numpy as np
from tmtools import tm_align

In [128]:
coords1 = pdb_numpy.Coor("src/pdb_numpy/tests/input/1u85.pdb")
coords2 = pdb_numpy.Coor("src/pdb_numpy/tests/input/1ubd.pdb")

coords1_ca = coords1.select_atoms("name CA")
coords2_ca = coords2.select_atoms("name CA")

seq1 = coords1.get_aa_seq()['A']
seq2 = coords2.get_aa_seq()['C']
res = tm_align(coords1_ca.xyz, coords2_ca.xyz, seq1, seq2)

In [131]:
%timeit tm_align(coords1_ca.xyz, coords2_ca.xyz, seq1, seq2)

4.22 ms ± 22.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [129]:
coords2.get_aa_seq()

{'C': 'TIACPHKGCTKMFRDNSAMRKHLHTHGPRVHVCAECGKAFVESSKLKRHQLVHTGEKPFQCTFEGCGKRFSLDFNLRTHVRIHTGDRPYVCPFDGCNKKFAQSTNLKSHILTHA'}

In [130]:
res.u

array([[-0.75530653, -0.5759304 ,  0.31275584],
       [ 0.17315727,  0.28488949,  0.94279082],
       [-0.63208275,  0.76625202, -0.11545235]])